In [1]:
import os, sys,pytz
import django
from datetime import datetime
sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/kebadiretsemotlhabi/source/flourish-project/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/kebadiretsemotlhabi/miniconda3/envs/flourish/lib/python3.11/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/kebadiretsemotlhabi/miniconda3/envs/flourish/lib/python3.11/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/kebadiretsemotlhabi/miniconda3/envs/flourish/lib/python3.11/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/kebadiretsemotlhabi/miniconda3/envs/flourish/lib/python3.11/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to

Loading Data Encryption ...
 * found encryption keys in /Users/kebadiretsemotlhabi/source/flourish-project/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 4 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.child

Remove mother from the c_sec_quart1_schedule1 schedule and put her in the c_sec_fu_quart1_schedule1 schedule,which aligns with the child ,then link the mothers schdule to the child

In [2]:
from edc_visit_schedule.models import SubjectScheduleHistory

subject_identifier = 'B142-040990112-0'
schedule_name_previous = 'c_sec_quart1_schedule1'

subject_history_obj = SubjectScheduleHistory.objects.get(
subject_identifier=subject_identifier,
schedule_name = schedule_name_previous
)

onschedule_model_cls= django_apps.get_model(subject_history_obj.onschedule_model)

onschedule_model_obj = onschedule_model_cls.objects.get(
    subject_identifier=subject_identifier,
    schedule_name=schedule_name_previous,
    onschedule_datetime=subject_history_obj.onschedule_datetime,
)

onschedule_model_obj.delete()
subject_history_obj.delete()

(1, {'edc_visit_schedule.SubjectScheduleHistory': 1})

In [5]:
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from flourish_caregiver.models import OnScheduleCohortCSecSeq,MaternalVisit
from flourish_child.models.onschedule import OnScheduleChildCohortCSecSeq
import pytz
tz = pytz.timezone('Africa/Gaborone')

maternal_visit = MaternalVisit.objects.get(subject_identifier=subject_identifier,visit_code='3000B')
child_schedule = OnScheduleChildCohortCSecSeq.objects.get(subject_identifier='B142-040990112-0-10')
report_datetime = maternal_visit.report_datetime
report_datetime = report_datetime.astimezone(tz)
schedule_name='c_sec_fu_quart1_schedule1'

_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model='flourish_caregiver.onschedulecohortcsecseq',name=schedule_name
)
schedule.put_on_schedule(
    subject_identifier=subject_identifier,onschedule_datetime=report_datetime,
    schedule_name=schedule_name,base_appt_datetime=report_datetime
)
print("Schedule created for",schedule)
onschdule_mother= OnScheduleCohortCSecSeq.objects.get(subject_identifier=subject_identifier)
onschdule_mother.child_subject_identifier =child_schedule.subject_identifier
onschdule_mother.save()

print("Linked with child")




Schedule created for c_sec_fu_quart1_schedule1
Linked with child
